# Les arbres Phylogénétiques

Les arbres phylogénétiques sont largement utilisés en biologie et notamment en génétique. Ils sont constitués de branches se séparant en n&oelig;ud et se terminant en feuilles. Les arbres sont des examples de graphes orientés acycliques. C'est à dire qu'ils ont une direction et ne possède pas de circuit.

Les arbres phylogénétiques sont utilisés pour représenter des relations de parenté entres des êtres vivants ou des groupes d'êtres vivants. Chaque n&oelig;ud montrant l'ancêtre commun des différentes branches subsidiaires. La hauteur d'un n&oelig;ud est la longueur du plus grand chemin depuis une feuille jusqu'à ce n&oelig;ud. La profondeur d'un n&oelig;ud est la distance avec la racine.

Ici nous utiliserons des arbres en format Newick (.nwk), c'est l'un des formats les plus couramment utilisé. Ces arbres contiennent l'information des n&oelig;ud de l'arbre, de leur hiérarchie ainsi que la distance de chacun avec son n&oelig;ud d'ordre supérieur dans un réseau de parenthèses imbriquées.

# La librairie ETE

ETE est une librairie python qui facilite la manipulation, analyse et visualisation d'arbres phylogénétiques. L'avantage d'ETE est qu'il ne fait aucune distinction entre un arbre et un n&oelig;ud. Les arbres pouvant être représentés à partir de leur racine, un n&oelig;ud peut donc devenir la racine d'un arbre. Il est donc très facile de partitionner ou de fusionner des arbres ETE.

Cependant il existe des arbres phylogénétiques non enracinés, ce n'est pas un problème car ETE considèrera le pointeur du n&oelig;ud ayant la plus grande hauteur comme la racine. Ainsi un arbre non enraciné sera un arbre dont la racine a plus que deux enfants.

In [1]:
from ete3 import Tree
unrooted_tree = Tree( "(A,B,(C,D));" )
print(unrooted_tree)

D:\Software\Miniconda\lib\site-packages\ete3-3.1.2-py3.7.egg\ete3\evol\parser\codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?
D:\Software\Miniconda\lib\site-packages\ete3-3.1.2-py3.7.egg\ete3\evol\parser\codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?



   /-A
  |
--|--B
  |
  |   /-C
   \-|
      \-D


In [2]:
rooted_tree = Tree( "((A,B),(C,D));" )
print(rooted_tree)


      /-A
   /-|
  |   \-B
--|
  |   /-C
   \-|
      \-D


Vous pouvez voir que j'ai ici importé la librairie ETE grâce à `import ete3`. Pour l'installer vous pouvez utiliser l'outil de gestion des paquets de votre choix (conda, mamba ou bien pip).

Concernant la génération des arbres phylogénétiques ETE propose une suite de toolkits donnant notamment accès à des commandes bash pour la génération d'arbres. Cependant leur installation est complexe et pose de nombreux problèmes de compatibilité, nous ne vous recommandons donc pas son utilisation. Si vous êtes toutefois interéssés par cela un guide rapide vous est proposé en fin de tuto.

# Nos modules

Nous remplacerons les toolkits d'ETE des modules utilisant principalement la librairie Biopython. Vous les trouverez dans les fichiers joints sous les noms de `sequences_alignment.py` et `tree_generation.py`.

Le premier module sert à faire de l'aligment multiple de séquences tout en gérer plusieurs cas possibles.

In [4]:
import sequences_alignment
help(sequences_alignment)

Help on module sequences_alignment:

NAME
    sequences_alignment - This module align a multi sequences fasta file.

DESCRIPTION
    ClustalW and Muscle algorithms are both supported for the alignment.

FUNCTIONS
    align_seq(in_dir, method='clustalw', out_file=False)
        Align multiple sequences using the MUSCLE algorithm.
        
        Parameters
        ----------
        in_dir : dir
            The directory to the file containing the sequences.
        method : str (default clustalw)
            The alignment algorithm, either clustalw or muscle.
        out_dir : boolean (default False)
            Wether or not a output file should be generated.
        
        Returns
        -------
        Bio.Align.MultipleSeqAlignment
            The aligned sequences.

FILE
    d:\data\fac\master\m1\python\ete_tuto\sequences_alignment.py




Cependant son fonctionnement peut se résummer à l'appel de la command line bash suivante :
Windows :
```Bash
./clustalo.exe -i seq_in.fasta -o seq_out.fasta --force
```
Linux :
```Bash
clustalo -i seq_in.fasta -o seq_out.fasta --force
```
`--force` est utilisé pour forcer l'écrasement du fichier de sortie si il existe déjà.

Le deuxième module s'occupe de la génération d'arbre phylogénétiques. Ces arbres nécéssitent des séquences alignées, il fait donc appel au module précédent si vous n'avez pas préalablement aligné les séquences d'entrée.

In [5]:
import tree_generation
help(tree_generation)

Help on module tree_generation:

NAME
    tree_generation - This module generate trees based on a multi sequences alignment or does perform the alignment in case it has not been preiviously done.

DESCRIPTION
    See the documentation of the sequences_alignment module for the alignment.
    
    NJ and UPGMA algorithms are both supported for the tree generation.

FUNCTIONS
    draw_tree(tree, method)
        Save a phylogenetic tree as a svg file.
        
        Parameters
        ----------
        tree : Bio.Phylo.BaseTree.Tree
            The phylogenetic tree to save.
        method : str
            The method used to generate the tree, used for the file name.
    
    parsimony_constructor(tree, seq, svg=False)
        Construct a tree using the parsimony method.
        
        Parameters
        ----------
        tree : Bio.Phylo.BaseTree.Tree
            A model tree to use as comparison.
        seq : Bio.Align.MultipleSeqAlignment
            A multiple sequences alignem

# Le jeu de donnée

Pour mettre en pratique les outils d'ETE nous étudierons un jeu séquences de la protéine Spike du SARS-COV2. Ce jeu de donnée à été récupéré depuis la base de donnée du NCBI. Afin de trier ces séquences un threshold du nombre d'apparition de ces séquences a appliqué. Cela nous donne le fichier fasta que vous trouverez en `data\spike_data_708.fasta`

Le fichier fasta à ensuite été aligné à l'aide du module `sequences_alignment` présenté ci-dessus et en utilisant Clustal$\Omega$. Un arbre a ensuite été généré à partir de cet alignement et grâce au module `tree_generation`. Nous avons utilisé blosum90 comme matrice de substitution à cause de la proximité des séquences. Nous avons utilisé le Neighbor Joining comme méthode phénétique.